sarah: make 10,000 the threshold face area that would not encounter change
below code is about the moving forwards and backwards
todo: increase tolerance and threshold.

In [4]:
import cv2
import numpy as np


# Load the cascade for face detection
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

# Start the video capture
cap = cv2.VideoCapture(0)

# Set the initial position of the motor
initial_position = 0

desired_face_area = 0
current_face_area = 0
callibrated = False
moving = False

while True:
    
    # Read the frame
    _, frame = cap.read()

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray,scaleFactor=1.2, minNeighbors=4, minSize=(30, 30), flags=cv2.CASCADE_SCALE_IMAGE)

    areas=[]
    if np.any(faces): # faces is not empty
        # find the largest face 
        if (len(faces)>1): # if there are more than one face
            for (x, y, w, h) in faces:
                areas.append((x+w)*(y+h))

            maxArea = max(areas)
            maxAreaPos = areas.index(maxArea)

            face = faces[maxAreaPos] #largest face
        elif (len(faces)==1):
            face = faces[0]
    
        (x,y,w,h) = face 

        # Draw a rectangle around the faces
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

        # Get the center of the face
        face_center_x = x + w/2
        face_center_y = y + h/2
        
        # field of view center
        frame_center_x = frame.shape[1]/2
        frame_center_y = frame.shape[0]/2

        # calculate the area of the desired face rectangle
        current_area = (x+w)*(y+h)
        
        # Calculate the error from the center of the frame
        error_x = frame_center_x - face_center_x
        error_y = frame_center_y - face_center_y
        
        print("error_x")
        print(error_x)
        
        # left and right motion
        if (error_x>10): #TODO: include tolerances
            print("turn right") 
            moving = True
        elif (error_x<10):
            print("turn left") #todo: directions might be wrong
            moving = True
        else:
            moving = False
        
        # callibrate
        if cv2.waitKey(1) & 0xFF == ord('b'):
            desired_face_area = current_area
            callibrated = True
            
        print("desired_face_area")
        print(desired_face_area)

        print("current_faca_area")
        print(current_area)
        
        if callibrated:
            if (current_area - desired_face_area > 100000): #TODO: can change the tolerance
                print("zoom out")
                moving = True
            elif (current_area - desired_face_area<-100000):
                print("zoom in")
                moving = True
            else:
                moving = False
        else:
            print ("not callibrated")
        
        if not moving:
            print ("stop")

    else: #faces empty
        print("faces empty")

    # Show the frame
    cv2.imshow("Face Tracking", frame)

    # Exit if the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture
cap.release()

# Close all windows
cv2.destroyAllWindows()

faces empty
error_x
42.0
turn right
desired_face_area
0
current_faca_area
1139645
not callibrated
error_x
10.5
turn right
desired_face_area
0
current_faca_area
1174896
not callibrated
error_x
-4.0
turn left
desired_face_area
0
current_faca_area
1185048
not callibrated
error_x
-11.5
turn left
desired_face_area
0
current_faca_area
1203384
not callibrated
error_x
-12.5
turn left
desired_face_area
0
current_faca_area
1191680
not callibrated
error_x
-2.5
turn left
desired_face_area
0
current_faca_area
1169774
not callibrated
error_x
12.5
turn right
desired_face_area
0
current_faca_area
1174326
not callibrated
error_x
6.0
turn left
desired_face_area
0
current_faca_area
1200388
not callibrated
error_x
9.0
turn left
desired_face_area
0
current_faca_area
1202740
not callibrated
error_x
2.0
turn left
desired_face_area
0
current_faca_area
1193130
not callibrated
error_x
1.5
turn left
desired_face_area
0
current_faca_area
1168136
not callibrated
error_x
-4.0
turn left
desired_face_area
0
current_f

error_x
-45.5
turn left
desired_face_area
0
current_faca_area
1301806
not callibrated
error_x
-41.0
turn left
desired_face_area
0
current_faca_area
1300995
not callibrated
error_x
-46.5
turn left
desired_face_area
0
current_faca_area
1305135
not callibrated
error_x
-46.0
turn left
desired_face_area
0
current_faca_area
1297224
not callibrated
error_x
-49.5
turn left
desired_face_area
0
current_faca_area
1305135
not callibrated
error_x
-47.0
turn left
desired_face_area
0
current_faca_area
1309731
not callibrated
error_x
-44.5
turn left
desired_face_area
0
current_faca_area
1303288
not callibrated
error_x
-49.5
turn left
desired_face_area
0
current_faca_area
1304908
not callibrated
error_x
-48.0
turn left
desired_face_area
0
current_faca_area
1306396
not callibrated
error_x
-49.0
turn left
desired_face_area
0
current_faca_area
1308694
not callibrated
error_x
-47.5
turn left
desired_face_area
0
current_faca_area
1296998
not callibrated
error_x
-48.0
turn left
desired_face_area
0
current_fa

error_x
45.0
turn right
desired_face_area
0
current_faca_area
984700
not callibrated
error_x
44.0
turn right
desired_face_area
0
current_faca_area
982401
not callibrated
error_x
43.0
turn right
desired_face_area
0
current_faca_area
1000184
not callibrated
error_x
37.5
turn right
desired_face_area
0
current_faca_area
1003392
not callibrated
error_x
36.0
turn right
desired_face_area
0
current_faca_area
1019640
not callibrated
error_x
32.0
turn right
desired_face_area
0
current_faca_area
1019358
not callibrated
error_x
26.5
turn right
desired_face_area
0
current_faca_area
1034876
not callibrated
error_x
27.5
turn right
desired_face_area
0
current_faca_area
1027840
not callibrated
error_x
32.0
turn right
desired_face_area
0
current_faca_area
1015000
not callibrated
error_x
31.0
turn right
desired_face_area
0
current_faca_area
1015875
not callibrated
error_x
38.0
turn right
desired_face_area
0
current_faca_area
1010904
not callibrated
error_x
41.0
turn right
desired_face_area
0
current_faca